<a href="https://colab.research.google.com/github/Faareh-Ahmed/Download_Satellite_Imagery/blob/main/SentinelTimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install sentinelhub


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.8/249.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.8 MB/s eta 0:00:00
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6085 sha256=6c33380c53f8a606ff4088fe894747fccfc575771ce777818174ef7eeafbe482
  Stored in directory: /root/.cache/pip/wheels/2f/a1/c8/543df0e8f5e824c3e92a432e32deb9cd89ae686095ee8cfcbe
Successfully built utm


# Downloading Sentinel Timeseries

In [2]:

CLIENT_ID = 'b88bf46a-f979-4fd1-90d2-9f182a9fefdb'
CLIENT_SECRET = 'eeNwFVeuG4CQwaKhkyhyijijWKuRlqog'

In [3]:
from sentinelhub import SHConfig

config = SHConfig()

config.sh_client_id = CLIENT_ID
config.sh_client_secret = CLIENT_SECRET

if not config.sh_client_id or not config.sh_client_secret:
  print('Please provide credentials to use Process API.')

In [4]:
from sentinelhub import (
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
)


In [5]:

coords = (73.027177, 33.685496, 73.083129, 33.731193)

resolution = 10

bbox = BBox(bbox=coords, crs=CRS.WGS84)
image_size = bbox_to_dimensions(bbox, resolution=resolution)

print(f'Image shape at {resolution} m resolution: {image_size}')

Image shape at 10 m resolution: (528, 497)


In [25]:
import datetime

start = datetime.datetime(2018, 1, 1)
end = datetime.datetime(2024, 12, 31)
n_chunks = 150 #no of images to download
tdelta = (end - start) / n_chunks
edges = [(start + i * tdelta).date().isoformat() for i in range(n_chunks)]
slots = [(edges[i], edges[i + 1]) for i in range(len(edges) - 1)]

print("Monthly time windows:\n")
for slot in slots:
    print(slot)

Monthly time windows:

('2018-01-01', '2018-01-18')
('2018-01-18', '2018-02-04')
('2018-02-04', '2018-02-21')
('2018-02-21', '2018-03-10')
('2018-03-10', '2018-03-27')
('2018-03-27', '2018-04-13')
('2018-04-13', '2018-04-30')
('2018-04-30', '2018-05-17')
('2018-05-17', '2018-06-03')
('2018-06-03', '2018-06-20')
('2018-06-20', '2018-07-07')
('2018-07-07', '2018-07-24')
('2018-07-24', '2018-08-10')
('2018-08-10', '2018-08-27')
('2018-08-27', '2018-09-13')
('2018-09-13', '2018-09-30')
('2018-09-30', '2018-10-17')
('2018-10-17', '2018-11-03')
('2018-11-03', '2018-11-20')
('2018-11-20', '2018-12-07')
('2018-12-07', '2018-12-24')
('2018-12-24', '2019-01-10')
('2019-01-10', '2019-01-27')
('2019-01-27', '2019-02-13')
('2019-02-13', '2019-03-03')
('2019-03-03', '2019-03-20')
('2019-03-20', '2019-04-06')
('2019-04-06', '2019-04-23')
('2019-04-23', '2019-05-10')
('2019-05-10', '2019-05-27')
('2019-05-27', '2019-06-13')
('2019-06-13', '2019-06-30')
('2019-06-30', '2019-07-17')
('2019-07-17', '2019

In [26]:
evalscript = """
//VERSION=3

function setup() {
  return {
    input: ["B02", "B03", "B04"],
    output: { bands: 3 }
  };
}

function evaluatePixel(sample) {
  return [2.5 * sample.B04, 2.5 * sample.B03, 2.5 * sample.B02];
}
"""

def get_true_color_request(time_interval):
    return SentinelHubRequest(
    evalscript=evalscript,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L2A,
            time_interval=('2020-11-11', '2024-10-15'),
            other_args={"dataFilter": {"mosaickingOrder": "leastCC"}}
        ),
    ],
    responses=[
        SentinelHubRequest.output_response('default', MimeType.TIFF),
    ],
    bbox=bbox,
    size=image_size,
    config=config
)

In [27]:
# create a list of requests
list_of_requests = [get_true_color_request(slot) for slot in slots]
list_of_requests = [request.download_list[0] for request in list_of_requests]

# download data with multiple threads
data = SentinelHubDownloadClient(config=config).download(list_of_requests, max_threads=5)

In [20]:
# Step 1: Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [28]:
output_folder = '/content/drive/MyDrive/sentinelhub_Sentinel2_l2a/'
import os
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [29]:
import matplotlib.pyplot as plt
import numpy as np

# some stuff for pretty plots
# ncols = 5
# nrows = 10
# aspect_ratio = image_size[0] / image_size[1]
# subplot_kw = {"xticks": [], "yticks": [], "frame_on": False}

# fig, axs = plt.subplots(ncols=ncols, nrows=nrows, figsize=(5 * ncols * aspect_ratio, 5 * nrows), subplot_kw=subplot_kw)

for idx, image in enumerate(data):
    # ax = axs[idx // ncols][idx % ncols]
    # ax.imshow(np.clip(image * 2.5 / 255, 0, 1))
    # ax.set_title(f"{slots[idx][0]}  -  {slots[idx][1]}", fontsize=10)

    # Save each individual image to the specified Google Drive folder
    image_filename = f"{output_folder}image_{idx+1}_{slots[idx][0]}_to_{slots[idx][1]}.png"
    plt.imsave(image_filename, np.clip(image * 2.5 / 255, 0, 1))

# plt.tight_layout()

# Downloading LadSat TimeSeries

In [49]:

coords = (73.027177, 33.685496, 73.083129, 33.731193)

resolution = 30

bbox = BBox(bbox=coords, crs=CRS.WGS84)
image_size = bbox_to_dimensions(bbox, resolution=resolution)

print(f'Image shape at {resolution} m resolution: {image_size}')

Image shape at 30 m resolution: (176, 166)


In [50]:
import datetime

start = datetime.datetime(2018, 1, 1)
end = datetime.datetime(2024, 12, 31)
n_chunks = 150 #no of images to download
tdelta = (end - start) / n_chunks
edges = [(start + i * tdelta).date().isoformat() for i in range(n_chunks)]
slots = [(edges[i], edges[i + 1]) for i in range(len(edges) - 1)]

print("Monthly time windows:\n")
for slot in slots:
    print(slot)

Monthly time windows:

('2018-01-01', '2018-01-18')
('2018-01-18', '2018-02-04')
('2018-02-04', '2018-02-21')
('2018-02-21', '2018-03-10')
('2018-03-10', '2018-03-27')
('2018-03-27', '2018-04-13')
('2018-04-13', '2018-04-30')
('2018-04-30', '2018-05-17')
('2018-05-17', '2018-06-03')
('2018-06-03', '2018-06-20')
('2018-06-20', '2018-07-07')
('2018-07-07', '2018-07-24')
('2018-07-24', '2018-08-10')
('2018-08-10', '2018-08-27')
('2018-08-27', '2018-09-13')
('2018-09-13', '2018-09-30')
('2018-09-30', '2018-10-17')
('2018-10-17', '2018-11-03')
('2018-11-03', '2018-11-20')
('2018-11-20', '2018-12-07')
('2018-12-07', '2018-12-24')
('2018-12-24', '2019-01-10')
('2019-01-10', '2019-01-27')
('2019-01-27', '2019-02-13')
('2019-02-13', '2019-03-03')
('2019-03-03', '2019-03-20')
('2019-03-20', '2019-04-06')
('2019-04-06', '2019-04-23')
('2019-04-23', '2019-05-10')
('2019-05-10', '2019-05-27')
('2019-05-27', '2019-06-13')
('2019-06-13', '2019-06-30')
('2019-06-30', '2019-07-17')
('2019-07-17', '2019

In [51]:
evalscript = """
//VERSION=3

function setup() {
  return {
    input: ["B02", "B03", "B04"],
    output: { bands: 3 }
  };
}

function evaluatePixel(sample) {
  return [2.5 * sample.B04, 2.5 * sample.B03, 2.5 * sample.B02];
}
"""

def get_true_color_request(time_interval):
    return SentinelHubRequest(
    evalscript=evalscript,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.LANDSAT_OT_L1,
            time_interval=('2020-11-11', '2024-10-15'),
            other_args={"dataFilter": {"mosaickingOrder": "leastCC"}}
        ),
    ],
    responses=[
        SentinelHubRequest.output_response('default', MimeType.TIFF),
    ],
    bbox=bbox,
    size=image_size,
    config=config
)

In [52]:
# create a list of requests
list_of_requests = [get_true_color_request(slot) for slot in slots]
list_of_requests = [request.download_list[0] for request in list_of_requests]

# download data with multiple threads
data = SentinelHubDownloadClient(config=config).download(list_of_requests, max_threads=5)

In [53]:
output_folder = '/content/drive/MyDrive/sentinelhub_Landsat_ot11/'
import os
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [54]:
import matplotlib.pyplot as plt
import numpy as np

# some stuff for pretty plots
# ncols = 5
# nrows = 10
# aspect_ratio = image_size[0] / image_size[1]
# subplot_kw = {"xticks": [], "yticks": [], "frame_on": False}

# fig, axs = plt.subplots(ncols=ncols, nrows=nrows, figsize=(5 * ncols * aspect_ratio, 5 * nrows), subplot_kw=subplot_kw)

for idx, image in enumerate(data):
    # ax = axs[idx // ncols][idx % ncols]
    # ax.imshow(np.clip(image * 2.5 / 255, 0, 1))
    # ax.set_title(f"{slots[idx][0]}  -  {slots[idx][1]}", fontsize=10)

    # Save each individual image to the specified Google Drive folder
    image_filename = f"{output_folder}image_{idx+1}_{slots[idx][0]}_to_{slots[idx][1]}.png"
    plt.imsave(image_filename, np.clip(image * 2.5 / 255, 0, 1))

# plt.tight_layout()